## Part One: Example

In the following we will work over the field $\mathbb{F}_{p^2}$ with $p = 45319$ and $i^2 = -1$ and consider the following curves:

$$
E_1 : y^2 = x^3 + 11x + 14i, \quad E_2 : y^2 = x^{3} + 2647 x + 4519 i 
$$

These two curves are two isogenous. We are going to use Sage to explore this statement and look at various ways we can look at the unknown degree two isogeny between them.

### First Steps: Define what we know

First up, let's take the information above and code this in Sage. First we will need to construct the field $\mathbb{F}_{p^2}$ and then from this, the curves themselves. 

In [ ]:
p = 45319
F = GF(p^2, name="i", modulus=x^2 + 1)
i = F.gen()
E1 = EllipticCurve(F, [11, 14*i])
E2 = EllipticCurve(F, [2647, 4519*i])

print(f"{F = }")
print(f"{E1 = }")
print(f"{E2 = }")

### Showing Curves are Isogenous

Before finding the isogeny itself, let us convince ourselves that these isogenies are in fact isogenous. First lets compute the order of each curve:

In [ ]:
print(f"{E1.order() = }")
print(f"{E2.order() = }")

The orders match! Things are looking good. In fact, Sage even has the following function:

In [ ]:
E1.is_isogenous(E2)

### Showing Curves are 2-Isogenous

To show that these curves are two isogenous, let's look at the modular polynomials. This first means that we must compute the j-invariant of each curve which is done with a simple call:

In [ ]:
j1 = E1.j_invariant()
j2 = E2.j_invariant()
print(f"{j1 = }")
print(f"{j2 = }")

### Aside: how to install modular polynomials

The modular polynomials themselves are not quite easily available (this should come in 10.3), but we can install what we need easily.

First, from your terminal type the following:

```bash
sage -i kohel_database
```

You should see a lot of output which ends with

```
[sagelib-10.2] real	0m22.519s
[sagelib-10.2] user	0m15.029s
[sagelib-10.2] sys	0m5.821s

real	0m28.477s
user	0m18.791s
sys	0m7.028s
Sage build/upgrade complete!
```

Then, you should be able to access the modular polynomials with the following commands

In [ ]:
PHI = ClassicalModularPolynomialDatabase()
PHI_2 = PHI[2]
PHI_3 = PHI[3]
print(f"{PHI_2 = }\n")
print(f"{PHI_3 = }")

Now we can look at whether the j-invariants $j_1$ and $j_2$ are roots of the polynomials:

In [ ]:
print(f"{PHI_2(j0=j1, j1=j2) = }")
print(f"{PHI_3(j0=j1, j1=j2) = }")

We see that as $\Phi_2(j_1, j_2) = 0$ then $E_1$ and $E_2$ are 2-isogenous but as $\Phi_3(j_1, j_2) \neq 0$, they are not 3-isogenous.

### Computing Points of Order Two

Now we know that our curves are two isogenous, we can attempt to find a kernel generator. To begin this, let's look at how we can find points of order two. Sage allows the computation of the abelian group with the following call

In [ ]:
E1.abelian_group()

and the generators of this group can be computed from:

In [ ]:
P, Q = E1.gens()
print(f"{P = }, {P.order() = }")
print(f"{Q = }, {Q.order() = }")

From these generators, points of order two can be computed from:

In [ ]:
P2 = (45320 // 2) * P
Q2 = (45320 // 2) * Q
R2 = P2 + Q2

print(f"{P2 = }, {P2.order() = }")
print(f"{Q2 = }, {Q2.order() = }")
print(f"{R2 = }, {R2.order() = }")

Another way of computing these is from finding the roots of the division polynomial:

In [ ]:
psi = E1.division_polynomial(2)
xs = psi.roots(multiplicities=False)
print(f"{xs = }")

Points can be computed from an x-coordinate by lifting them onto the curve:

In [ ]:
x1, x2, x3 = xs
print(f"{E1.lift_x(x1) = } ")
print(f"{E1.lift_x(x2) = } ")
print(f"{E1.lift_x(x3) = } ")

### Computing 2-isogenies

Given the points of order two computed above, we can compute an isogeny from these points with the following calls:

In [ ]:
phi_P = E1.isogeny(P2)
phi_Q = E1.isogeny(Q2)
phi_R = E1.isogeny(R2)

print(f"{phi_P = }\n")
print(f"{phi_Q = }\n")
print(f"{phi_R = }\n")

We can access the codomain of these isogenies with the `.codomain()` function. The question is are any of the above codomains isomorphic to the starting curve $E_2$.

In [ ]:
EP = phi_P.codomain()
EQ = phi_Q.codomain()
ER = phi_R.codomain()

print(f"{E2.is_isomorphic(EP) = }")
print(f"{E2.is_isomorphic(EQ) = }")
print(f"{E2.is_isomorphic(ER) = }")

We have found that the point $P_2$ generates the unknown isogeny linking $E_1$ and $E_2$!

### An Alternative Method

Another way to enumerate prime degree isogenies is available to us, which does all the work we did above in a single call

In [ ]:
E1.isogenies_prime_degree(2)

We could then do the same as the above by looping through these isogenies:

In [ ]:
kernel_poly = None
for phi in E1.isogenies_prime_degree(2):
    E_test = phi.codomain()
    if E2.is_isomorphic(E_test):
        kernel_poly = phi.kernel_polynomial()
        print(f"{kernel_poly = }")

Using the kernel polynomial we can also compute this isogeny using Kohel's algorithm:

In [ ]:
E1.isogeny(kernel_poly)

### Computing the Isogeny

Above we computed the isogeny up to isomorphism, but we can also compute the isomorphism itself to compute the precise isogeny we need. 

In [ ]:
print(f"{E2 = }")
print(f"{EP = }")
print(f"{E2.is_isomorphic(EP) = }")
EP.isomorphism_to(E2)

Now, to compute the isogeny we can compse $\phi_P$ with the above isomorphism:

In [ ]:
iso = EP.isomorphism_to(E2)
phi = iso * phi_P
print(f"{phi = }")

### Dual Isogeny

Given the isogeny $\phi : E_1 \to E_2$ we can also compute the dual isogeny $\hat{\phi} : E_2 \to E_1$ with a simple call:

In [ ]:
phi_dual = phi.dual()
phi_dual